In [3]:
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.io.shapereader as shpreader
from matplotlib.offsetbox import AnchoredOffsetbox, TextArea, VPacker
from matplotlib.lines import Line2D
from matplotlib import colors as c
import datetime
import os
import pandas as pd
import numpy as np

def generate_drought_outlook_map(ncname, shp_path, output_path, map_type='1_month'):
    """
    Generate a drought outlook map.
    
    :param ncname: Path to the NetCDF file
    :param shp_path: Path to the shapefile
    :param output_path: Path to save the output map
    :param map_type: '1_month' or '3_months'
    """
    # Open the NetCDF file
    ds = xr.open_dataset(ncname)
    
    # Get the 2D data array for outlook
    outlook_data = ds['outlook'].values
    lat = ds['lat'].values
    lon = ds['lon'].values
    
    # Define color levels and corresponding category colors
    colors = ['#FFFFFF', '#016838', '#5BB75B', '#FCDE66', '#B079D1', '#59207D']
    categories = ['No Drought', 'Drought Removed', 'Drought Improves', 'Drought Develops', 'Drought Persists', 'Drought Worsens']
    
    # Create a ListedColormap
    cmap = c.ListedColormap(colors)
    
    # Create the figure and axes
    fig, ax = plt.subplots(figsize=(12, 10), subplot_kw=dict(projection=ccrs.PlateCarree()))
    
    # Plot the pseudocolor map
    im = ax.pcolormesh(lon, lat, outlook_data, cmap=cmap, shading='auto')
    
    # Load and add shapefile geometries
    shp_feat = shpreader.Reader(shp_path).geometries()
    ax.add_geometries(shp_feat, ccrs.PlateCarree(), facecolor='none', linewidth=0.3)
    
    # Create custom legend
    legend_entries = [Line2D([0], [0], marker='s', color='none', label=category, markerfacecolor=color, markersize=10, linestyle='None') for color, category in zip(colors, categories)]
    ax.legend(handles=legend_entries, loc='upper right', title='Drought Outlook', framealpha=0, bbox_to_anchor=(1, 1), bbox_transform=ax.transAxes, borderaxespad=0.5)
    
    # Add title and subtitle
    current_date = datetime.datetime.now()
    current_month_name = current_date.strftime("%B")
    current_year = current_date.year
    
    if map_type == '1_month':
        subtitle_text = f"{current_month_name} {current_year}"
    else:
        end_month = (current_date + datetime.timedelta(days=60)).strftime("%B")
        subtitle_text = f"{current_month_name} {current_year} - {end_month} {current_year}"
    
    title = TextArea('Drought Outlook', textprops=dict(color='black', size=24, weight='bold'))
    subtitle = TextArea(subtitle_text, textprops=dict(color='black', size=18))
    anchored_box = AnchoredOffsetbox(loc='lower left', child=VPacker(children=[title, subtitle], align='left', pad=5, sep=5),
                                     frameon=False, bbox_to_anchor=(0.08, 0.06), bbox_transform=ax.transAxes,
                                     borderpad=0.1)
    ax.add_artist(anchored_box)
    
    # Add watermark
    ax.text(0.5, 0.5, 'Prototype', transform=ax.transAxes, fontsize=80, color='gray', alpha=0.2, ha='center', va='center', rotation=4)
    
    # Save the plot
    plt.savefig(output_path, dpi=300, bbox_inches='tight', pad_inches=0.1)
    plt.show()
    plt.close(fig)

def generate_rainfall_forecast_map(ncname, shp_path, output_path):
    """
    Generate a rainfall forecast map.
    
    :param ncname: Path to the NetCDF file
    :param shp_path: Path to the shapefile
    :param output_path: Path to save the output map
    """
    # Open the NetCDF file
    ds = xr.open_dataset(ncname)
    
    # Get the rainfall forecast data
    rain_data = ds['percentage_of_ensembles'].values[1, 0, :, :]  # Assuming we want the second bin and first time step
    lat = ds['latitude'].values
    lon = ds['longitude'].values
    
    # Get the forecast time
    forecast_time = ds['time'].values[0]
    forecast_date = pd.to_datetime(forecast_time)
    forecast_month = forecast_date.strftime('%B %Y')
    
    # Create the figure and axes
    fig, ax = plt.subplots(figsize=(12, 10), subplot_kw=dict(projection=ccrs.PlateCarree()))
    
    # Create a custom colormap
    colors = ['#a63603', '#e6550d', '#fdae6b', '#fee6ce', '#efedf5', '#bcbddc', '#807dba', '#4a1486']
    cmap = plt.cm.colors.ListedColormap(colors)
    bounds = [20, 30, 40, 50, 60, 70, 80, 90, 100]
    norm = plt.cm.colors.BoundaryNorm(bounds, cmap.N)
    
    # Plot the rainfall forecast data
    im = ax.pcolormesh(lon, lat, rain_data, cmap=cmap, norm=norm, shading='auto')
    
    # Add features
    ax.add_feature(cfeature.COASTLINE)
    ax.add_feature(cfeature.BORDERS)
    
    # Load and add shapefile geometries
    shp_feat = shpreader.Reader(shp_path).geometries()
    ax.add_geometries(shp_feat, ccrs.PlateCarree(), facecolor='none', edgecolor='black', linewidth=0.5)
    
    # Add colorbar
    cbar = plt.colorbar(im, ax=ax, orientation='vertical', pad=0.02, aspect=30, shrink=0.6)
    cbar.ax.tick_params(labelsize=10)
    
    # Add title and subtitle
    title = TextArea('Chance of exceeding the median rainfall', textprops=dict(color='black', size=16, weight='bold'))
    subtitle = TextArea(forecast_month, textprops=dict(color='black', size=14))
    anchored_box = AnchoredOffsetbox(loc='lower left', child=VPacker(children=[title, subtitle], align='left', pad=5, sep=5),
                                     frameon=False, bbox_to_anchor=(0.05, 0.05), bbox_transform=ax.transAxes,
                                     borderpad=0)
    ax.add_artist(anchored_box)
    
    # Set extent to focus on Australia
    ax.set_extent([110, 155, -45, -10], crs=ccrs.PlateCarree())
    
    # Save the plot
    plt.savefig(output_path, dpi=300, bbox_inches='tight', pad_inches=0.1)
    plt.close(fig)

def main():
    # Define paths
    base_path = "/Users/sabinmaharjan/projects/python/do/static"
    shp_path = os.path.join(base_path, "shapes/gadm36_AUS_1.shp")
    
    # Generate 1-month drought outlook map
    current_month = datetime.datetime.now().strftime("%m")
    current_year = datetime.datetime.now().year
    ncname_1month = os.path.join(base_path, f"result/nc/1_months/{current_month}_Final_{current_year}.nc")
    output_1month = os.path.join(base_path, f"result/maps/1_months/drought-outlook_1_{current_year}-{current_month}.jpg")
    generate_drought_outlook_map(ncname_1month, shp_path, output_1month, map_type='1_month')
    print(f"1-month drought outlook map saved: {output_1month}")

    # Generate 3-month drought outlook map
    ncname_3month = os.path.join(base_path, f"result/nc/3_months/app{current_month}_Final_{current_year}.nc")
    output_3month = os.path.join(base_path, f"result/maps/3_months/drought-outlook_3_{current_year}-{current_month}.jpg")
    generate_drought_outlook_map(ncname_3month, shp_path, output_3month, map_type='3_months')
    print(f"3-month drought outlook map saved: {output_3month}")

    # Generate rainfall forecast map
    ncname_rainfall = os.path.join(base_path, "file/forecast_updated.nc")
    output_rainfall = os.path.join(base_path, f"result/maps/forecast/rainfall_forecast_{current_year}-{current_month}.jpg")
    generate_rainfall_forecast_map(ncname_rainfall, shp_path, output_rainfall)
    print(f"Rainfall forecast map saved: {output_rainfall}")

if __name__ == "__main__":
    main()

1-month drought outlook map saved: /Users/sabinmaharjan/projects/python/do/static/result/maps/1_months/drought-outlook_1_2024-10.jpg
3-month drought outlook map saved: /Users/sabinmaharjan/projects/python/do/static/result/maps/3_months/drought-outlook_3_2024-10.jpg
Rainfall forecast map saved: /Users/sabinmaharjan/projects/python/do/static/result/maps/forecast/rainfall_forecast_2024-10.jpg
